In [8]:
import pandas as pd

df = pd.read_csv('test_dataset.csv')
df

,ch,ch_SC
0,而在中時報系以不堪虧損為由捨棄晚報的同時，另方面卻持續入股中天電視台，並有意在未來收購中視，...,而在中时报系以不堪亏损为由舍弃晚报的同时，另方面却持续入股中天电视台，并有意在未来收购中视，...
1,終戰後的十餘年間，可說是歌仔戲的黃金時代，人才輩出，除了活躍於戲院舞台的「內台戲」外，還有「...,终战后的十余年间，可说是歌仔戏的黄金时代，人才辈出，除了活跃於戏院舞台的「内台戏」外，还有「...
2,「國民美術」以非學院派美術基調的發展過程，巧妙地與 1998 年國內社區大學興起的「解放知識...,「国民美术」以非学院派美术基调的发展过程，巧妙地与 1998 年国内社区大学兴起的「解放知识...
3,汐止社大主任潘英海表示：「國民美術像集體參與的美術豐年祭，劉秀美在社區成立畫會，建構集體記憶...,汐止社大主任潘英海表示：「国民美术像集体参与的美术丰年祭，刘秀美在社区成立画会，建构集体记忆...
4,例如日治時期祖父在金瓜石經營「鈔利搗礦場」的鄭炯輝，將當時用水車淘洗金砂的過程，一一用圖畫記...,例如日治时期祖父在金瓜石经营「钞利捣矿场」的郑炯辉，将当时用水车淘洗金砂的过程，一一用图画记...
...,...,...
310911,誰來對抗資本主義？,谁来对抗资本主义？
310912,在西畫荒漠中披荊斬棘,在西画荒漠中披荆斩棘
310913,我怕！」,我怕！」
310914,多麼主觀傲慢！,多么主观傲慢！


In [9]:
sc_to_tc_table = pd.read_csv('簡化字對照標準字總表.csv')
sc_to_tc_table

,簡體字,正體字,非對稱簡繁字
0,卜,"['卜', '蔔']",True
1,叉,['叉'],False
2,一,['一'],False
3,又,['又'],False
4,口,['口'],False
...,...,...,...
4632,钻,['鑽'],False
4633,箩,['籮'],False
4634,銮,['鑾'],False
4635,蛮,['蠻'],False


In [10]:
import torch

from transformers import LlamaTokenizer
from peft import AutoPeftModelForCausalLM # PEFT = Parameter-Efficient Fine-Tuning

model_name = "/home/wirl/ytc/chinese-alpaca-2-7b_自己下載的"

def init_model(ft_num):
    adapter_model_path = f"/home/wirl/ytc/要寫的論文研究/code/qlora/output/checkpoint-{ft_num}"
    # merged_model_path = "/home/wirl/ytc/要寫的論文研究/code/results/chinese-alpaca-2-7b_merged_model_by_qlora/"

    print(f"Using ft_num={ft_num} model.")

    # https://github.com/artidoro/qlora/issues/29#issuecomment-1737072311
    # 請放棄 merge model，乖乖用 AutoPeftModelForCausalLM 接住 qlora.py 訓練的 adapter
    model = AutoPeftModelForCausalLM.from_pretrained(
        adapter_model_path,
        local_files_only=True, 
        load_in_4bit=True, 
    )
    return model

def get_response(tokenizer, model, prompt_template, sentence_text, remove_input=True):
    device = "cuda:0"
    full_prompt = prompt_template.format(sentence_text)

    # temperature, top_p, and top_k are only active when do_sample=True
    # if you set Top-k to 10, the LLM will only consider the 10 most probable next words. 
    # This will result in more fluent text, but it will also reduce the diversity of the text. 
    # TOP_K = 30
    TOP_K = 50
    # TOP_K = 70
    # If you set Top-p to 0.9, the LLM will only generate words that have a probability of at least 0.9. 
    # This will result in more diverse text, but it could also result in less fluent text.
    TOP_P = 0.95
    # TOP_P = 1.0

    # TEMP = 1.0 # don't specify

    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)

    # outputs = model.generate(**inputs, max_new_tokens=len(sentence_text))
    outputs = model.generate(
        **inputs, 
        max_new_tokens=len(sentence_text), 
        do_sample=True, 
        # temperature=TEMP,
        top_k=TOP_K, 
        top_p=TOP_P, 
        num_return_sequences=1,
    ) # for translation, https://huggingface.co/docs/transformers/tasks/translation#inference

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if remove_input:
        # 從 generate 出來的 output 中刪除 input text 的部分
        cleaned_output = decoded_output.replace(full_prompt, "")
        return cleaned_output
    else:
        return decoded_output



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/wirl/anaconda3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so


/home/wirl/anaconda3/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/wirl/anaconda3/lib/libcudart.so'), PosixPath('/home/wirl/anaconda3/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


CUDA SETUP: CUDA runtime path found: /home/wirl/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/wirl/anaconda3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


In [11]:
FT_NUM = 1250

tokenizer = LlamaTokenizer.from_pretrained(model_name, local_files_only=True, legacy=True)
model = init_model(FT_NUM)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Using ft_num=1250 model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
seed = 42
torch.manual_seed(seed)
model.config.seed = seed

In [12]:
sentence = df.loc[0, 'ch_SC']

translate_prompt_template = """
### Instruction:
翻譯成繁體中文: {}
### Response:
"""

print(get_response(tokenizer, model, translate_prompt_template, sentence))
print('---')

 而在中時報系以不堪虧損為由捨棄週報同時，另方面卻持續入股中天電視臺，並有意在未來收購中視，成就跨媒體集團霸業。
---


In [21]:
count = 0
while count < 10:
    print(get_response(tokenizer, model, translate_prompt_template, sentence))
    print('---')
    count += 1

而在中時報系以不堪虧損為由捨棄晚報同時，另方面卻持續入股中天電視台，並有意在未來收購中視，成就跨媒體集團霸業。
---
 而在中時報系以不堪虧損為由捨棄日報同時，另方面卻持續入股中天電視台，並有意於未來收購中視，成就跨媒體集團霸業。
---
而在中時報系以不堪虧損為由捨棄週刊同時，另方面卻持續入股中天電視台，並有意在未來收購中視，成就跨媒體集團霸業。
---
而在中時報系以不堪虧損為由捨棄週報同時，另方面卻持續入股中天電視臺，並有意在未來收購中視，成就跨媒體集團霸業。
---
而在中時報系以不堪虧損為由捨棄週刊雜誌同時，另方面卻持續入股中天電視台，並有意在未來收購中視，成就跨媒體集團霸業。
---
而在中時報系以不堪虧損為由捨棄晚報同時，另方面卻持續入股中天電視臺，並有意未來收購中視，成就跨媒體集團霸業。
---
 而在中時報系以不堪虧損為由捨棄週刊雜誌同時，另方面卻持續入股中天電視臺，並有意於未來收購中視，成就跨媒體集團霸業。
---
而在中時報系以不堪虧損為由捨棄週刊同時，另方面卻持續入股中天電視台，並有意未來收購中視，成就跨媒體集團霸業。
---
而在中時報系以不堪虧損為由捨棄週報同時，另方面卻持續入股中天電視台，並有意在未來收購中視，成就跨媒體集團霸業。
---
而在中報系以不堪虧損為由捨棄週刊同時，另方面卻持續入股中天電視台，並有意於未來收購中視，成就跨媒體集團霸業。
---


In [22]:
sentence1 = "台中后里位在台湾南北的交会点，隐藏着许多全国知名的景点。"
sentence2 = "我干什么不干你事。"
sentence3 = "我发现太后的头发很干燥。"
sentence4 = "芋头发芽了。"
sentence5 = "再坐在电脑前面 我头发都没了T_T。"
sentence6 = "他觉得丑时人生的通常都比较丑。"

In [23]:
count = 0
while count < 10:
    print(get_response(tokenizer, model, translate_prompt_template, sentence1))
    print('---')
    count += 1

 臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
 臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
 臺中后里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
 臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
 臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
 臺中后里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
 臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---
 臺中後里位在臺灣南北的交會點，隱藏著許多全國知名的景點。
---


In [13]:
# sentence = "而在中时报系以不堪亏损为由舍弃晚报的同时，另方面却持续入股中天电视台，并有意在未来收购中视，成就跨媒体集团霸业。"
asymmetric_words_in_sentence = set(sc_to_tc_table[sc_to_tc_table['非對稱簡繁字'] == True]['簡體字']) & set(sentence)

sc_to_tc_table[sc_to_tc_table['簡體字'].isin(asymmetric_words_in_sentence)]

,簡體字,正體字,非對稱簡繁字
174,台,"['台', '臺', '颱']",True
355,并,"['并', '並', '併']",True
629,系,"['系', '係', '繫']",True
961,舍,"['舍', '捨']",True
1388,面,"['面', '麵']",True


In [17]:
e = "而在中时报[系]以不堪亏损为由[舍]弃晚报的同时，另方[面]却持续入股中天电视[台]，[并]有意在未来收购中视，成就跨媒体集团霸业。"

prompt_template = """
### Instruction:
被[]框住的字是非對稱簡繁字，你必須根據上下文來推斷該如何翻譯此字，可能的字在->後面的[]裡，
[台]->[台, 臺, 颱], 并->[并, 並, 併], 系->[系, 係, 繫], 舍->[舍, 捨], 面->[面, 麵]
翻譯成繁體中文: 而在中时报[系]以不堪亏损为由[舍]弃晚报的同时，另方[面]却持续入股中天电视[台]，[并]有意在未来收购中视，成就跨媒体集团霸业。
### Response:
"""

print(get_response(tokenizer, model, prompt_template, e))
print('---')

[台]->[臺, 臺, 颱], 並->[並, 併, 併]，[係]->[係, 係]，[相]->[相, 相]
---


In [18]:
e = "而在中时报[系]以不堪亏损为由[舍]弃晚报的同时，另方[面]却持续入股中天电视[台]，[并]有意在未来收购中视，成就跨媒体集团霸业。"

prompt_template = """
### Instruction:
被[]框住的字是非對稱簡繁字，你必須根據上下文來推斷該如何翻譯此字，可能的字在->後面的[]裡，
翻譯成繁體中文: 而在中时报[系][系, 係, 繫]以不堪亏损为由[舍]->[舍, 捨]弃晚报的同时，另方[面]->[面, 麵]却持续入股中天电视[台]->[台, 臺, 颱]，[并]->[并, 並, 併]有意在未来收购中视，成就跨媒体集团霸业。
### Response:
"""

print(get_response(tokenizer, model, prompt_template, e))
print('---')

 在中時報[係, 係, 係]以不堪虧損為由[捨]->[捨, 捨, 捨]棄週報同時，另方[面]->[面, 麵, 面]卻持續入股中天電視[台]->[臺, 
---


In [ ]:
import jieba

from rouge import Rouge
import nltk.translate.gleu_score as gleu
from nltk.translate.bleu_score import sentence_bleu

# 可參考
src = list(jieba.cut("上層漏水耍手段不去處理可以怎麼做"))
ref = list(jieba.cut("上層漏水耍手段不去處理可以怎麼做"))

# ref: https://github.com/gcunhase/NLPMetrics

score_gleu = gleu.sentence_gleu([ref], src, min_len=1, max_len=2)
print(score_gleu)

In [ ]:
import pandas as pd

import nltk.translate.gleu_score as gleu
from nltk.translate.bleu_score import sentence_bleu

true_sentence = "而在中時報系以不堪虧損為由捨棄晚報的同時，另方面卻持續入股中天電視台，並有意在未來收購中視，成就跨媒體集團霸業。"
revised_response = "而在中時報系以不堪虧損為由捨棄週報雜誌的同時，另方面卻持續入股中天電視台，並有意在未來收購中視，成就跨媒體集團霸業。"

ref = list(jieba.cut(true_sentence))
src = list(jieba.cut(revised_response))

print('---')
print(f"Ground truth sentence:     {true_sentence}")
print(f"Model translated sentence: {revised_response}")
print('---')
print(f"Ground truth sentence (cutted):     {ref}")
print(f"Model translated sentence (cutted): {src}")
print('---')

bleu_score = sentence_bleu([ref], src)
print(f"bleu_score: {bleu_score}")
print('---')

gleu_score = gleu.sentence_gleu([ref], src, min_len=1, max_len=2)
print(f"gleu_score: {gleu_score}")
print('---')

